In [22]:
import pandas as pd
import sqlite3
from custom_functions import *

In [24]:
conn = sqlite3.connect('../../data/processed/main.db')
query_df = '''SELECT * FROM step3_final_df'''
query_performance = '''SELECT * FROM step3_performance_metrics'''
df_final = pd.read_sql(query_df, conn, index_col='index').reset_index(drop=True)
performance_metrics = pd.read_sql(query_performance, conn, index_col='index').reset_index(drop=True)

In [32]:
x = list(df_final.drop('SalePrice_log', axis=1).columns)
model, _ = produce_model(df_final, x, 'SalePrice_log')
print(model.summary())

Modeling: SalePrice_log ~ Heating_ElecBB+Heating_FloorWall+Heating_HeatPump+Heating_HotWater+Heating_Radiant+SqFtTotLiving_log+Basement_Finished+Porch_Open+Porch_Closed+Porch_Both
                            OLS Regression Results                            
Dep. Variable:          SalePrice_log   R-squared:                       0.404
Model:                            OLS   Adj. R-squared:                  0.403
Method:                 Least Squares   F-statistic:                     1217.
Date:                Fri, 12 Mar 2021   Prob (F-statistic):               0.00
Time:                        17:55:47   Log-Likelihood:                -9173.4
No. Observations:               17986   AIC:                         1.837e+04
Df Residuals:                   17975   BIC:                         1.845e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef  